# INF367 Assignment 2, Exercise 2
Author: Johanna Jøsang (fak006)

In [1]:
import numpy as np
import random
from IPython.display import Image
import timeit

In [2]:
# Arguments
n = 10 #number of variables
delta = 0.1
eps = 0.4
eta = 0.01

ten_div_delta = int(10/delta)
k = 1000
N = 40*k

## Notes about methods used for PAC implementation with noise
The target is generated in the exact same manner as the previous assignment, so the description of it is the same.
Since a hypothesis only can be given positive examples if the target is satisfiable, only satisfiable targets are used in this implementation. A satisfiable target is generated by combining data generated using a normal distribution. "1" indicates that a variable is evaluated to True, "0" that is is evaluated to False. The target does not necessarily contain all literals, hence it may contain "3" which indicated that this variable is not present in the target. \
Target generation example:\
[0, 1, 1, 0, 0, 0, 1, 1, 0, 0] <- basis 1 and \
[0, 1, 1, 0, 1, 0, 0, 0, 0, 1] <- basis 2 generate:\
[0, 1, 1, 0, 3, 0, 3, 3, 0, 3] <- target

Examples are generated in the same way as bases for the target, by normal distribution. Each example is categorized by whether it satisfies the target or not. With a probability of eta, the examples will be misclassified. The following examples are based on the target given above. Notice that the valuation of a variable not present in the target has no influence on the categorization.

([0, 1, 1, 1, 0, 0, 1, 1, 0, 1],  0) <- valuation that doesn't satisfy target \
([0, 1, 1, 0, 1, 0, 1, 0, 0, 0],  1) <- valuation that does satisfy target \
([0, 1, 0, 1, 0, 0, 1, 1, 0, 1],  1) <- valuation that doesn't satisfy target, but is misclassified 

Since the algorithm for learning with noise needs to keep track of statistical information for each literal, the hypothesis is represented slightly differently than in my previous assignment. Each literal gets it's own position in the array, so that information about it, such as the p0-value, can be stored there.

![title](literals_representation.png)

After training, the hypothesis would look something like this:
[0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1], where each "1" represents the presence of a certian literal. So if we translate this to the normal representation of literals, we get:
[0, 1, 1, 0, 3, 0, 1, 0, 0, 0]

If we compare the target and the final hypothesis we can see the result. \
Target-----: [0, 1, 1, 0, 3, 0, 3, 3, 0, 3] \
Trained hyp: [0, 1, 1, 0, 3, 0, 1, 0, 0, 0] 

The hypothesis has not quite reached the target.

In [3]:
def generate_uncategorized_data(size, dimensions):
    """
    Using a random normal distribution, 2D datasets of 1s and 0s are generated
    
    :param size: number of arrays to generate
    :param dimensions: length of each array
    :return : 2D array of uncategorized data
    
    """
    normal_distribution = np.random.normal(size=[size, dimensions]) >= 0
    normal_distribution_ints = normal_distribution.astype(int)                        
    return normal_distribution_ints.tolist()

def generate_satisfiable_target(size, dimensions):
    """
    Using a random normal distribution, 2D datasets of 1s and 0s are generated
    
    :param size: number of arrays to generate
    :param dimensions: length of each array
    
    """
    target_basis_ints = generate_uncategorized_data(size, dimensions)
    target = [1]*(len(target_basis_ints[0]))
    for j in range (len(target_basis_ints[0])):
        column_target_value = 3
        column_bool = target_basis_ints[0][j]
        use_bool = True
        for i in range (len(target_basis_ints)):
            if (target_basis_ints[i][j] != column_bool):
                column_target_value = 3
                use_bool = False
                continue
        if(use_bool):
            if(column_bool == True):
                column_target_value = 1
            else:
                column_target_value = 0
        target[j] = column_target_value#
    return target

def categorize_data(valuation, target):
    """
    Categorizes valuations based on if they satisfy the target   
    
    :param valuation: valuation to categorize
    :param target: used classify the valuation
    :return : 1 if the valuation satisfies the target, 0 otherwise
    
    """
    if(len(valuation) != len(target)):
        print('data length:', str(len(valuation)), ' != target length', str(len(target)))
        return 4 # this should never happen, hence and invalid number is given
    for i in range (len(target)):
        #if t[i] = 3, d[i] = 2 or t[i] == d[i], d entails t
        if((target[i] != 3) and (valuation[i] != 2) and (target[i] != valuation[i])):
                return 0 # valuation does not satisfy target
    return 1 # valuation does satisfy target

def categorize_data_w_noise(valuation, target):
    """
    Categorizes valuations based on if they satisfy the target. 
    With a probability of eta, the categorization will be wrong.
    
    :param valuation: valuation to categorize
    :param target: used classify the valuation
    :return : 1 if the valuation satisfies the target, 0 otherwise
    
    """
    if(len(valuation) != len(target)):
        print('data length:', str(len(valuation)), ' != target length', str(len(target)))
        return 4 # this should never happen, hence and invalid number is given
    for i in range (len(target)):
        #if t[i] = 3, d[i] = 2 or t[i] == d[i], d entails t
        misclassify = random.random() <= eta #determine whether classification should be correct or not
        if((target[i] != 3) and (valuation[i] != 2) and (target[i] != valuation[i])):
            if misclassify:
                return 1
            else:
                return 0 # valuation does not satisfy target
    if misclassify:
        return 0
    else:
        return 1 # valuation does satisfy target


def generate_noisy_examples(data_list, target):
    """
    Generates a list of tupples where each set of valuations is paired with a classification, ie a positive or negative example.
    
    :param data: set of uncategorized examples
    :param data: 
    :return : list of categorized valuations
    """
    examples = []
    for valuation in data_list:
        category = categorize_data_w_noise(valuation, target)
        examples.append((valuation, category))
    return examples

def disjoint_union(mt, mh):
    """
    Generated the disjoint union between two sets.
    
    :param mt: set of valuations that satisfy the target
    :param mh: det of valuations that satisfy the hypothesis
    :return : disjoint union between mt and mh
    """
    for e in mt:
        if e in mh:
            mt.remove(e)
            mh.remove(e)
    for e in mh:
        if e in mt:
            mt.remove(e)
            mh.remove(e)
    return mt + mh

def calculate_t_hat(S, target, hypothesis):
    """
    Calculates the estimated error of the trained hypothesis w.r.t the target. 
    
    :param S: set of unlabeled examples
    :param target: to estimate the error w.r.t
    :param hypothesis: to estimate the error of
    """
    mt = [] #set of valuations in S that satisfy the target
    mh = [] #set of valuations in S that satisfy the target
    for s in S:
        if (categorize_data(s, target) == 1):
            mt.append(s)
        if (categorize_data(s, hypothesis) == 1):
            mh.append(s)
    error_s = disjoint_union(mt, mh) #find misclassified valuations
    t_hat = len(error_s) / len(S) # error = (no. misclassified valuations)/(total no. of valuations)
    return t_hat


def get_all_positive_examples(examples):
    """
    Returns all examples that are categorized as positive:
    
    :param examples: list of examples to select the positive ones from
    :return pos_examples: list of the positively categorized examples
    """
    pos_examples = []
    for e in examples:
        if e[1] == 1:
            pos_examples.append(e)
    return pos_examples

def translate_binary_literals_to_binary_variables(lit_list):
    """
    Translates a hypotheis representation from literals to hypothesis representation with variables.
    
    :param lit_list: set of binary representation of literals
    : return bin_variables: representation of hypothesis in the form of variables
    """
    bin_variables = np.full(n, 3)
    for i in range(len(bin_variables)):
        if lit_list[2*(i-1)] == 1:
            bin_variables[i] = 1
        elif lit_list[2*(i-1)+1] == 1:
            bin_variables[i] = 0
        else:
             bin_variables[i] = 3
        #if (lit_list[2*(i-1)] == 1) and (lit_list[2*(i-1)+1] == 1):
            #print("Error: variable in hypothesis at index", i, " is both true and false")
            #print(lit_list)
    return bin_variables

def get_p0(uncategorized_data):
    """
    Generates an array of p_0(z) for all literals z.
    
    :param uncategorized_data: set of uncategorized examples.
    :return p0: array of p_0(z) for all literals z
    """
    p0_list = np.zeros(2*n)
    for e in uncategorized_data:
        for i in range(n):
            if e[i] == 0:
                p0_list[2 * i] += 1
            elif e[i] == 1:
                 p0_list[(2 * i)+1] += 1
            else:
                print("Error: valuation must be either zero or 1", e)
    p0 = p0_list / k
    #print("p0", p0)
    return p0

def get_set_I(p0):
    """
    Generates the set I.
    
    :param p0: array of p_0(z) for all literals z
    :return set_I: array represenation of the set I
    """
    set_I = np.zeros(2*n)
    for i in range(2*n):
        if p0[i] >= eps/(32*(2*n)**2):
            set_I[i] = 1
    return set_I

def get_p01(examples):
    """
    Generates an array of p_01(z) for all literals z.
    
    :param examples: array of categorized examples.
    :return p01: array of p_01(z) for all literals z
    """
    pos_ex = get_all_positive_examples(examples)
    p01_count = np.zeros(2*n)
    for e in pos_ex:
        for i in range(n):
            if e[0][i] == 0:
                p01_count[2 * i] += 1
            elif e[0][i] == 1:
                 p01_count[(2 * i)+1] += 1
    p01 = p01_count / k
    return p01

def get_apr(p0, p01):
    """
    Generates an array of apr(z) for all literals z.
    
    :param p0, p01: array of p0(z) and p01(z)
    :return apr: array of apr(z) for all literals z. 
    """
    apr = np.zeros(2*n)
    for i in range(2*n):
        apr[i] = p01[i]/p0[i]
    return apr

def get_number_of_neg_examples(examples):
    """
    Counts the number of negatively classified examples.
    """
    number_of_neg_ex = 0
    for e in examples:
        if e[1] == 0:
            number_of_neg_ex += 1
    return number_of_neg_ex

def get_noise_estimate(examples, set_I, apr):
    """
    Finds the best estimate for noise, by following the approach in 
    "Learning from Noisy Examples" by Angluin et al.
    
    :param examples: array of categorized examples.
    :param set_I: the set of significant literals
    :return final_noise_estimate: best estimate for noise
    """
    number_of_neg_examples = get_number_of_neg_examples(examples)
    noise_estimate_1 = number_of_neg_examples/k
    noise_estimate_2 = 1
    for i in range(2*n):
        if set_I[i] == 1:
            if apr[i] < noise_estimate_2:
                noise_estimate_2 = apr[i]
    final_noise_estimate = min(noise_estimate_1, noise_estimate_2)
    return final_noise_estimate

def generate_hypothsis_from_noisy_examples(set_I, apr, noise_estimate):
    """
    Using the algorithm described in "Learning from Noisy Examples" by Angluin et al.
    the method trains up a hypothesis with noisy examples.
    
    :param set_I: the set of significant literals
    :param apr: array with apr(z) for each literal z
    :param  noise_estimate: estimate for noise
    """
    final_hyp_literals = np.zeros(2*n)
    for i in range(2*n):
        if set_I[i] == 1: #if the literal is in I
            if apr[i] <= (noise_estimate + (eps * (1 - 2 * eta) / (4 * (2*n)))):
                final_hyp_literals[i] = 1
    final_hyp = translate_binary_literals_to_binary_variables(final_hyp_literals)
    return final_hyp

## PAC implementation

In [11]:
# Print out arguments being used
print('Number of variables used:', n)
print('Epsilon:', eps)
print('Delta:', delta)
print('Eta_b:', eta)
print('k:', k)
print()

target = generate_satisfiable_target(2,n) # we generate a target with 2 bases
uncategorized_data = generate_uncategorized_data(k, n)
examples = generate_noisy_examples(uncategorized_data, target)
#calculate p0(z) for all literals
p0 = get_p0(uncategorized_data)
#Generate the set I
set_I = get_set_I(p0)
#Calculate p01 for all literals
p01 = get_p01(examples)
#Calculate apr(z) for all literals
apr = get_apr(p0, p01)
#Get estimate for noise
noise_estimate = get_noise_estimate(examples, set_I, apr)
trained_hypothesis = generate_hypothsis_from_noisy_examples(set_I, apr, noise_estimate)
S = generate_uncategorized_data(N, n)
t = calculate_t_hat(S, target, trained_hypothesis) #calculate the error
print("Normal distribution used for generating training set.")
print("Target:", target)
print("Trained hyp:", trained_hypothesis)
print("Error:", t)

Number of variables used: 10
Epsilon: 0.4
Delta: 0.1
Eta_b: 0.01
k: 1000

Normal distribution used for generating training set.
Target: [3, 3, 0, 3, 3, 0, 3, 3, 3, 3]
Trained hyp: [3 3 3 0 3 3 0 3 3 3]
Error: 0.378925


## Generalization guarantee
Through a bit of trial and error, I ended up choosing eta_b = 0.01 and k = 4200 for my algorithm.

Unfortunately the runtime for the large k required for epsilon = 0.3 on my computer was 30min+. I let it run for a couple of different values of k and got these results:

| k      | bad error rate |
| ----------- | ----------- |
| 1000      | 0.35|
| 2000      | 0.17 |
| 3000      | 0.13|
| 4000      | 0.1    |
| 4100      | 0.8   |
| 4200      | 0.5   |
| 4300   | 0.7  |
| 4500   | 0.12  |

So it seemed 4200 is a good value for k in this implementation of the algorithm.

I have now changed epsilon to 0.04, where I found that k = 1000 is enough to get a bad error rate less than or equal to 0.1. It still takes a couple of minutes to run for me, but hopefully your computer is faster than mine and it will only take a few seconds of your time.

In [12]:
# Print out arguments being used
print('Number of variables used:', n)
print('Epsilon:', eps)
print('Delta:', delta)
print('Eta_b:', eta)
print('k:', k)
print('10/delta:', ten_div_delta)
print('N:', N)
print()

print("Number of iterations:", ten_div_delta)
bad_error_count = 0 # number of errors larger than epsilon
start = timeit.default_timer()
for i in range(ten_div_delta):
    print("Currently on interation", i)
    target = generate_satisfiable_target(2,n) # we generate a target with 2 bases
    uncategorized_data = generate_uncategorized_data(k, n)
    examples = generate_noisy_examples(uncategorized_data, target)
    #calculate p0(z) for all literals
    p0 = get_p0(uncategorized_data)
    #Generate the set I
    set_I = get_set_I(p0)
    #Calculate p01 for all literals
    p01 = get_p01(examples)
    #Calculate apr(z) for all literals
    apr = get_apr(p0, p01)
    #Get estimate for noise
    noise_estimate = get_noise_estimate(examples, set_I, apr)
    trained_hypothesis = generate_hypothsis_from_noisy_examples(set_I, apr, noise_estimate)
    S = generate_uncategorized_data(N, n)
    t = calculate_t_hat(S, target, trained_hypothesis) #calculate the error
    if (t > eps):
        bad_error_count = bad_error_count + 1 # count the number of errors greater than epsilon
print('Number of errors greater than epsilon:', bad_error_count)
print('Bad error rate:', bad_error_count/ten_div_delta)
stop = timeit.default_timer()
print('Runtime for k', k, " is:", stop - start, "seconds")  

Number of variables used: 10
Epsilon: 0.4
Delta: 0.1
Eta_b: 0.01
k: 1000
10/delta: 100
N: 40000

Number of iterations: 100
Currently on interation 0
Currently on interation 1
Currently on interation 2
Currently on interation 3
Currently on interation 4
Currently on interation 5
Currently on interation 6
Currently on interation 7
Currently on interation 8
Currently on interation 9
Currently on interation 10
Currently on interation 11
Currently on interation 12
Currently on interation 13
Currently on interation 14
Currently on interation 15
Currently on interation 16
Currently on interation 17
Currently on interation 18
Currently on interation 19
Currently on interation 20
Currently on interation 21
Currently on interation 22
Currently on interation 23
Currently on interation 24
Currently on interation 25
Currently on interation 26
Currently on interation 27
Currently on interation 28
Currently on interation 29
Currently on interation 30
Currently on interation 31
Currently on interation